# Advanced NumPy

## `numpy` internals

In [1]:
import numpy as np
np.random.seed(2374)

In [2]:
arr = np.random.randint(10, size=(8,8))

Information about array elements:

In [3]:
arr.itemsize, arr.dtype

(4, dtype('int32'))

In [5]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

How to step through array memory? Using `strides` property:

In [7]:
arr.strides

(32, 4)

I. e. `arr[0, 1]` is 8 bytes away from `arr[0, 0]` (one step along axis `1`), while `arr[1, 0]` is 64 bytes away from `arr[0, 0]` (one step along axis `0`).

In [8]:
arr.strides[0] == arr.shape[1] * arr.itemsize

True

But what about views?

In [9]:
arr_view = arr[::2, 1:]

In [10]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [11]:
arr_view

array([[7, 6, 8, 2, 9, 2, 0],
       [6, 9, 3, 3, 5, 1, 2],
       [3, 9, 5, 2, 4, 5, 2],
       [8, 7, 4, 7, 3, 8, 2]])

Information about underlying array structure:

In [34]:
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [13]:
arr_view.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Views always have base array:

In [15]:
arr_view.base

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [16]:
arr.base

In [17]:
arr_view.base is arr

True

`strides` are provided with respect to the **underlying data** (which is the same between original array `arr` and view array `arr_view`!):

In [18]:
arr_view.strides

(64, 4)

In [19]:
arr_view.shape

(4, 7)

Since view is not contiguous, this relation is not True anymore:

In [20]:
arr_view.strides[0] == arr_view.shape[1] * arr_view.itemsize

False

Also, view starts not from byte 0 of the data, but steps 8 bytes inside the data:

In [21]:
np.byte_bounds(arr_view)[0] - np.byte_bounds(arr)[0]

4

In [22]:
np.byte_bounds(arr_view)

(2395943543684, 2395943543904)

In [23]:
np.byte_bounds(arr_view)[1] - np.byte_bounds(arr)[1]

-32

In [24]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [25]:
arr_view

array([[7, 6, 8, 2, 9, 2, 0],
       [6, 9, 3, 3, 5, 1, 2],
       [3, 9, 5, 2, 4, 5, 2],
       [8, 7, 4, 7, 3, 8, 2]])

In [26]:
arr_view.strides

(64, 4)

In [27]:
arr.T

array([[5, 3, 8, 0, 4, 8, 9, 0],
       [7, 1, 6, 7, 3, 4, 8, 3],
       [6, 5, 9, 5, 9, 8, 7, 6],
       [8, 8, 3, 7, 5, 7, 4, 6],
       [2, 7, 3, 1, 2, 8, 7, 4],
       [9, 5, 5, 2, 4, 9, 3, 0],
       [2, 3, 1, 8, 5, 2, 8, 9],
       [0, 1, 2, 1, 2, 2, 2, 4]])

In [29]:
arr.T.strides,arr.strides

((4, 32), (32, 4))

Transpose reports similar strides, is it a view?

In [31]:
arr_view.T.strides,arr_view.strides

((4, 64), (64, 4))

In [32]:
arr_view.T[::2, 1:].base is arr

True

### Creating views manually

In [35]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

Take memory, associated with `arr`:

In [36]:
arr.data

Create a new array, poiting to the same memory:

In [37]:
np.ndarray(buffer=arr.data, shape=arr.shape, dtype=arr.dtype)

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

Create a new *view* pointing to the same memory:

In [40]:
np.ndarray(buffer=arr.data, shape=(4, 8), dtype=arr.dtype, strides=(64, 8))

array([[5, 6, 2, 2, 3, 5, 7, 3],
       [8, 9, 3, 1, 0, 5, 1, 8],
       [4, 9, 2, 5, 8, 8, 8, 2],
       [9, 7, 7, 8, 0, 6, 4, 9]])

In [43]:
np.ndarray(buffer=arr.data, shape=(4, 8), dtype=arr.dtype, strides=(64, 8)).base is arr

True

## Cache effects

In [44]:
large_arr = np.random.randint(100, size=(1000000,))

In [46]:
larger_arr.shape, large_arr.shape

((4000000,), (1000000,))

In [47]:
%timeit -n 100 -r 3 large_arr.sum()

346 µs ± 44.5 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [50]:
%timeit -n 100 -r 3 larger_arr[::STEP].sum()

1.64 ms ± 73.7 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [55]:
del large_arr, larger_arr

In [56]:
large_arr = np.random.randint(100, size=(5, 10000000))

In [57]:
large_arr.nbytes // (1024*1024)

190

In [58]:
large_arr

array([[77, 70, 20, ..., 80, 44, 53],
       [97, 22, 75, ..., 78, 98, 74],
       [ 5, 49, 96, ..., 75, 39, 40],
       [11, 20, 60, ..., 51, 50, 43],
       [87, 67, 54, ..., 17, 42, 25]])

In [59]:
large_arr.T

array([[77, 97,  5, 11, 87],
       [70, 22, 49, 20, 67],
       [20, 75, 96, 60, 54],
       ...,
       [80, 78, 75, 51, 17],
       [44, 98, 39, 50, 42],
       [53, 74, 40, 43, 25]])

In [60]:
large_arr.T.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [61]:
large_arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [62]:
%timeit -n 50 -r 3 large_arr.sum(axis=1).sum(axis=0)

19.2 ms ± 56.8 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [65]:
%timeit -n 500 -r 3 large_arr.sum()

19.7 ms ± 194 µs per loop (mean ± std. dev. of 3 runs, 500 loops each)


In [63]:
%timeit -n 50 -r 3 large_arr.sum(axis=0).sum()

44.7 ms ± 218 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [66]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=0)

19.9 ms ± 180 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [67]:
large_arr.T.base is large_arr

True

## Mamory allocations in computations

How long does it take to create a copy?

In [68]:
%timeit -n 20 -r 3 large_arr.copy()

42.6 ms ± 408 µs per loop (mean ± std. dev. of 3 runs, 20 loops each)


Operations create new arrays as well:

In [69]:
%timeit -n 20 -r 3 large_arr + 1

62 ms ± 768 µs per loop (mean ± std. dev. of 3 runs, 20 loops each)


`np.add` and `+` do more or less the same:

In [70]:
%timeit -n 20 -r 3 np.add(large_arr, 1)

64.9 ms ± 2.63 ms per loop (mean ± std. dev. of 3 runs, 20 loops each)


But in-place operations are faster (no allocations):

In [71]:
%timeit -n 20 -r 3 np.add(large_arr, 1, out=large_arr)

23.3 ms ± 754 µs per loop (mean ± std. dev. of 3 runs, 20 loops each)


In [ ]:
large_arr

In [ ]:
np.add(large_arr, 1, out=large_arr)

# Broadcasting

How can we operate on arrays of different shapes? Should we reshape them first to a common shape?

In [72]:
arr_2d = np.random.randint(10, size=(10, 3))
arr_1d_1 = np.random.randint(10, size=(3, ))
arr_1d_2 = np.random.randint(10, size=(10, ))

In [73]:
arr_2d

array([[9, 5, 7],
       [4, 2, 6],
       [5, 5, 1],
       [8, 9, 9],
       [7, 0, 8],
       [5, 5, 1],
       [5, 7, 5],
       [6, 9, 6],
       [2, 9, 4],
       [3, 6, 1]])

In [74]:
arr_1d_1

array([5, 4, 2])

In [75]:
arr_1d_2

array([3, 1, 9, 7, 3, 1, 5, 4, 2, 8])

In [76]:
arr_2d, arr_1d_1

(array([[9, 5, 7],
        [4, 2, 6],
        [5, 5, 1],
        [8, 9, 9],
        [7, 0, 8],
        [5, 5, 1],
        [5, 7, 5],
        [6, 9, 6],
        [2, 9, 4],
        [3, 6, 1]]), array([5, 4, 2]))

Can we add the two?

In [77]:
arr_2d + arr_1d_1

array([[14,  9,  9],
       [ 9,  6,  8],
       [10,  9,  3],
       [13, 13, 11],
       [12,  4, 10],
       [10,  9,  3],
       [10, 11,  7],
       [11, 13,  8],
       [ 7, 13,  6],
       [ 8, 10,  3]])

But what was really added to `arr_2d`?

In [78]:
(arr_2d + arr_1d_1) - arr_2d

array([[5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2],
       [5, 4, 2]])

Can we do the same with `arr_1d_2`?

In [82]:
arr_2d + arr_1d_2

ValueError: operands could not be broadcast together with shapes (10,3) (10,) 

We need to change `arr_1d_2` shape first:

In [84]:
arr_2d + arr_1d_2.reshape((10,2))

ValueError: cannot reshape array of size 10 into shape (10,2)

Alternatively, we can do:

In [ ]:
np.expand_dims(arr_1d_2, axis=1)

In [ ]:
arr_2d + np.expand_dims(arr_1d_2, axis=1)

It seems `arr_1d_2` was "replicated" in the same way as `arr_1d_1` but along different axis:

In [ ]:
(arr_2d + np.expand_dims(arr_1d_2, axis=1)) - arr_2d

To reveal the pattern, let's try a `3D` array:

In [85]:
arr_3d = np.random.randint(10, size=(7, 10, 3))

In [86]:
arr_1d_1.shape

(3,)

In [ ]:
arr_3d

In [88]:
arr_3d + arr_1d_1-arr_3d

array([[[5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2]],

       [[5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2]],

       [[5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2]],

       [[5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2]],

       [[5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2],
        [5, 4, 2]],

       [[5, 4, 2],
        [5, 4, 2],
  

In [ ]:
(arr_3d + arr_1d_1) - arr_3d

In [89]:
arr_3d.shape, arr_1d_1.shape

((7, 10, 3), (3,))

Can we do the same with `arr_1d_2`?

In [90]:
arr_3d + arr_1d_2

ValueError: operands could not be broadcast together with shapes (7,10,3) (10,) 

In [91]:
arr_3d.shape, arr_1d_2.shape, np.expand_dims(arr_1d_2, axis=1).shape

((7, 10, 3), (10,), (10, 1))

In [ ]:
(arr_3d + np.expand_dims(arr_1d_2, axis=1)) - arr_3d

Broadcasting rules:
    
- All input arrays with `ndim` smaller than the input array of largest `ndim`, have 1’s **prepended** to their shapes.
- The size in each dimension of the output shape is the **maximum** of all the input sizes in that dimension.
- An input can be used in the calculation if its size in a particular dimension either **matches** the output size in that dimension, or **is exactly 1**.
- If an input has a dimension of size 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension. In other words, the stepping machinery of a `ufunc` will simply not step along that dimension (stride will be 0 for that dimension).

### How broadcasting really works

What happens, when we add a unit dimension somewhere?

In [92]:
arr_1d_1[np.newaxis, :]

array([[5, 4, 2]])

In [93]:
arr_1d_1[np.newaxis, :].strides

(0, 4)

`strides[0]` is `0`, which means we can use dimension `0` of `arr_1d_1[np.newaxis, :]` in any (underlying, C) loop with any number of iterations. Let's emulate this in pure Python:

In [94]:
arr_1d_1_bc = arr_1d_1[np.newaxis, :]
arr_1d_1_bc

array([[5, 4, 2]])

In [95]:
arr_2d

array([[9, 5, 7],
       [4, 2, 6],
       [5, 5, 1],
       [8, 9, 9],
       [7, 0, 8],
       [5, 5, 1],
       [5, 7, 5],
       [6, 9, 6],
       [2, 9, 4],
       [3, 6, 1]])

In [96]:
for i in range(arr_2d.shape[0]):

    print(f"Adding elements of row {i}")

    for j in range(arr_2d.shape[1]):
        arr_2d_address = arr_2d.strides[1] * j + arr_2d.strides[0] * i
        arr_1d_address = arr_1d_1_bc.strides[1] * j + arr_1d_1_bc.strides[0] * i

        print(f"\tarr_2d address: {arr_2d_address}")
        print(f"\tarr_1d_1_bc address: {arr_1d_address}")
    print("-" * 80)

Adding elements of row 0
	arr_2d address: 0
	arr_1d_1_bc address: 0
	arr_2d address: 4
	arr_1d_1_bc address: 4
	arr_2d address: 8
	arr_1d_1_bc address: 8
--------------------------------------------------------------------------------
Adding elements of row 1
	arr_2d address: 12
	arr_1d_1_bc address: 0
	arr_2d address: 16
	arr_1d_1_bc address: 4
	arr_2d address: 20
	arr_1d_1_bc address: 8
--------------------------------------------------------------------------------
Adding elements of row 2
	arr_2d address: 24
	arr_1d_1_bc address: 0
	arr_2d address: 28
	arr_1d_1_bc address: 4
	arr_2d address: 32
	arr_1d_1_bc address: 8
--------------------------------------------------------------------------------
Adding elements of row 3
	arr_2d address: 36
	arr_1d_1_bc address: 0
	arr_2d address: 40
	arr_1d_1_bc address: 4
	arr_2d address: 44
	arr_1d_1_bc address: 8
--------------------------------------------------------------------------------
Adding elements of row 4
	arr_2d address: 48
	arr_1